# Covert the Excel Tool to Python

In [2]:
import pandas as pd
import numpy as np

## Read file

### factors

In [7]:
dd = pd.read_excel('moves4-aadvmt-converter-tool-2023-08.xltx', sheet_name='Calc')
dd.head()

,HPMSVTypeID,monthID,noOfDays,Monthly Adjustment Factor,Weekend-Day Adjustment Factor,Weekday VMT,Weekend-Day VMT,Monthly Weekday VMT,Monthly Weekend-Day VMT,Total Monthly VMT,...,HPMSBaseYearVMT,Unnamed: 14,sourceTypeID,sourceTypeName,HPMSVtypeID,HPMSVtypeName,Unnamed: 19,HPMSVTypeID.2,monthID.1,MOVES3 Default Monthly Adjustment Factor
0,10,1,31,0.314422,1,0,0,0,0,0,...,0.0,NaN,11,Motorcycle,10,Motorcycles,NaN,10,1,0.314422
1,10,2,28,0.284347,1,0,0,0,0,0,...,0.0,NaN,21,Passenger Car,25,Light Duty Vehicles,NaN,10,2,0.284347
2,10,3,31,0.699932,1,0,0,0,0,0,...,0.0,NaN,31,Passenger Truck,25,Light Duty Vehicles,NaN,10,3,0.699932
3,10,4,30,1.208476,1,0,0,0,0,0,...,0.0,NaN,32,Light Commercial Truck,25,Light Duty Vehicles,NaN,10,4,1.208476
4,10,5,31,1.432673,1,0,0,0,0,0,...,0.0,NaN,41,Intercity Bus,40,Buses,NaN,10,5,1.432673


In [4]:
mon_factors = dd.iloc[:, -3:]   # select the last 3 columns

In [8]:
mon_factors = mon_factors.rename(columns={'HPMSVTypeID.2':'HPMSVTypeID', 'monthID.1':'monthID'})
mon_factors.head()

,HPMSVTypeID,monthID,MOVES3 Default Monthly Adjustment Factor
0,10,1,0.314422
1,10,2,0.284347
2,10,3,0.699932
3,10,4,1.208476
4,10,5,1.432673


### Num_of_Days

In [11]:
num_of_days = dd.iloc[:, 2]

In [13]:
# concate number of days and the facors dataset
reference_df = pd.concat([mon_factors,num_of_days],axis = 1)
reference_df.head(10)

,HPMSVTypeID,monthID,MOVES3 Default Monthly Adjustment Factor,noOfDays
0,10,1,0.314422,31
1,10,2,0.284347,28
2,10,3,0.699932,31
3,10,4,1.208476,30
4,10,5,1.432673,31
5,10,6,1.522898,30
6,10,7,1.599453,31
7,10,8,1.618592,31
8,10,9,1.358852,30
9,10,10,1.140123,31


## Gain Input

In [14]:
vtypes = set(reference_df['HPMSVTypeID'])
vtypes

{10, 25, 40, 50, 60}

In [15]:
reference_df.to_csv('default_factors_nofdays.csv')

In [16]:
# Here we use fake data first
year = 2017
vtype = 25
vAADVMT = 20000

## Time disaggregation

### Weekday &rarr; Month &rarr; Year

In [17]:
certain_type_df = reference_df.loc[mon_factors['HPMSVTypeID']==vtype]  # CERTAIN VEHICLE TYPE
certain_type_df = certain_type_df.sort_values('monthID')  # SORTED BY MONTH


In [18]:
# Get the months factors for certain vehicle types
ty_mon_factors = certain_type_df['MOVES3 Default Monthly Adjustment Factor']

In [19]:
certain_type_df.head()

,HPMSVTypeID,monthID,MOVES3 Default Monthly Adjustment Factor,noOfDays
12,25,1,0.877027,31
13,25,2,0.836551,28
14,25,3,0.980778,31
15,25,4,0.987626,30
16,25,5,1.050034,31


In [11]:

certain_type_df['day VMT'] = ty_mon_factors * vAADVMT   # AADVMT in certain month
certain_type_df['Monthly Weekday VMT'] = certain_type_df['day VMT'] * certain_type_df['noOfDays'] * (5/7)  # total vmt in weekday, in a month
certain_type_df['Monthly Weekend VMT'] = certain_type_df['day VMT'] * certain_type_df['noOfDays'] * (2/7)
certain_type_df['Monthly Total VMT'] = certain_type_df['Monthly Weekday VMT'] + certain_type_df['Monthly Weekend VMT']  # total vmt in a month
certain_type_df['Yearly Total VMT'] = np.sum(certain_type_df['Monthly Total VMT'])  # total vmt in a year


### Day &rarr; Hour

I don't why there are road types in the hourVMTFraction table. I am going to use the averaged fractions.

In [20]:
hour_fraction_orig = pd.read_excel('moves4-aadvmt-converter-tool-2023-08.xltx', sheet_name='hourVMTFraction')

Change the source type to vehicle type in the excel.

In [22]:
source_to_vtype = {
    11: 10,
    21: 25, 31: 25, 32: 25,
    41: 40, 42: 40, 43: 40,
    51: 50, 52: 50, 53: 50, 54: 50,
    61: 60, 62: 60
}

hour_fraction_orig['vtype'] = hour_fraction_orig.sourceTypeID.map(source_to_vtype)
hour_fraction_orig.head(10)

,sourceTypeID,roadTypeID,dayID,hourID,hourVMTFraction,vtype
0,11,1,2,1,0.021474,10
1,11,1,2,2,0.014443,10
2,11,1,2,3,0.010968,10
3,11,1,2,4,0.007495,10
4,11,1,2,5,0.006839,10
5,11,1,2,6,0.010359,10
6,11,1,2,7,0.018430,10
7,11,1,2,8,0.026812,10
8,11,1,2,9,0.036385,10
9,11,1,2,10,0.047541,10


**Calculate avgerade hourVMTFraction.**

In [26]:
test = hour_fraction_orig[hour_fraction_orig['roadTypeID'].isin([2, 3])]
test.head()

,sourceTypeID,roadTypeID,dayID,hourID,hourVMTFraction,vtype
48,11,2,2,1,0.016421,10
49,11,2,2,2,0.011192,10
50,11,2,2,3,0.008542,10
51,11,2,2,4,0.006793,10
52,11,2,2,5,0.007219,10


In [27]:
test.groupby(['vtype', 'dayID', 'hourID']).mean()

sourceTypeID  roadTypeID  hourVMTFraction
vtype dayID hourID                                           
10    2     1               11.0         2.5         0.016421
            2               11.0         2.5         0.011192
            3               11.0         2.5         0.008542
            4               11.0         2.5         0.006793
            5               11.0         2.5         0.007219
...                          ...         ...              ...
60    5     20              61.5         2.5         0.043986
            21              61.5         2.5         0.035731
            22              61.5         2.5         0.030743
            23              61.5         2.5         0.023852
            24              61.5         2.5         0.017318

[240 rows x 3 columns]

In [34]:
tt = test.groupby(['vtype', 'dayID', 'hourID']).mean().reset_index()
tt.head()

,vtype,dayID,hourID,sourceTypeID,roadTypeID,hourVMTFraction
0,10,2,1,11.0,2.5,0.016421
1,10,2,2,11.0,2.5,0.011192
2,10,2,3,11.0,2.5,0.008542
3,10,2,4,11.0,2.5,0.006793
4,10,2,5,11.0,2.5,0.007219


In [35]:
tt[tt['vtype'] == 10]['hourVMTFraction'].sum()

np.float64(2.00000014)

In [14]:
hour_fraction_orig.groupby(['vtype', 'dayID', 'hourID']).mean()

sourceTypeID  roadTypeID  hourVMTFraction
vtype dayID hourID                                           
10    2     1               11.0         3.0         0.019453
            2               11.0         3.0         0.013143
            3               11.0         3.0         0.009998
            4               11.0         3.0         0.007214
            5               11.0         3.0         0.006991
...                          ...         ...              ...
60    5     20              61.5         3.0         0.044230
            21              61.5         3.0         0.035560
            22              61.5         3.0         0.031392
            23              61.5         3.0         0.024506
            24              61.5         3.0         0.017671

[240 rows x 3 columns]

In [49]:
# save the file
hour_fraction = hour_fraction_orig.groupby(['vtype', 'dayID', 'hourID']).mean()
hour_fraction.to_csv('/uufs/chpc.utah.edu/common/home/haskins-group1/users/szhao/RoadSalt/AADVMT/MOVES_converter/hour_fraction.csv')

In [56]:
df2 = pd.read_csv('/uufs/chpc.utah.edu/common/home/haskins-group1/users/szhao/RoadSalt/AADVMT/MOVES_converter/hour_fraction.csv', index_col=[0,1])

np.float64(2.000000157999999)

In [45]:
hour_fraction

sourceTypeID  roadTypeID  hourVMTFraction
vtype dayID hourID                                           
10    2     1               11.0         3.0         0.019453
            2               11.0         3.0         0.013143
            3               11.0         3.0         0.009998
            4               11.0         3.0         0.007214
            5               11.0         3.0         0.006991
...                          ...         ...              ...
60    5     20              61.5         3.0         0.044230
            21              61.5         3.0         0.035560
            22              61.5         3.0         0.031392
            23              61.5         3.0         0.024506
            24              61.5         3.0         0.017671

[240 rows x 3 columns]

In [47]:
tt2 = hour_fraction.reset_index()
tt2[tt2['vtype'] == 10]['hourVMTFraction'].sum()

np.float64(2.000000158)

In [44]:
tt2.head(20)

,vtype,dayID,hourID,sourceTypeID,roadTypeID,hourVMTFraction
0,10,2,1,11.0,3.0,0.019453
1,10,2,2,11.0,3.0,0.013143
2,10,2,3,11.0,3.0,0.009998
3,10,2,4,11.0,3.0,0.007214
4,10,2,5,11.0,3.0,0.006991
5,10,2,6,11.0,3.0,0.010520
6,10,2,7,11.0,3.0,0.018130
7,10,2,8,11.0,3.0,0.026837
8,10,2,9,11.0,3.0,0.037295
9,10,2,10,11.0,3.0,0.049420
